## Convolutional NN to classify govuk content to level2 taxons

Based on:
https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

### Load requirements and data

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical, layer_utils, plot_model

from keras.layers import (Embedding, Input, Dense, Dropout, 
                          Activation, Conv1D, MaxPooling1D, Flatten, concatenate, Reshape)
from keras.models import Model, Sequential
from keras.optimizers import rmsprop
from keras.callbacks import TensorBoard, Callback, ModelCheckpoint
import keras.backend as K
from keras.losses import binary_crossentropy

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score 
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.utils import class_weight

import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

import functools

import h5py

from scipy import sparse
import time


### Environmental vars

In [ ]:
DATADIR = os.getenv('DATADIR')

### Print data version

In [ ]:
print('algorithm running on data extracted from content store on {}'.format(DATADIR))

## Hyperparameters

Intuition for POS_RATIO is that it penalises the prediction of zero for everything, which is attractive to the model because the multilabel y matrix is super sparse. 

Increasing POS_RATIO should penalise predicting zeros more.

In [ ]:
#MAX_NB_WORDS
MAX_SEQUENCE_LENGTH =1000
EMBEDDING_DIM = 100 # keras embedding layer output_dim = Dimension of the dense embedding
P_THRESHOLD = 0.5 #Threshold for probability of being assigned to class
POS_RATIO = 0.5 #ratio of positive to negative for each class in weighted binary cross entropy loss function
NUM_WORDS=20000 #keras tokenizer num_words: None or int. Maximum number of words to work with 
#(if set, tokenization will be restricted to the top num_words most common words in the dataset).

### Read in data

In [ ]:
train = np.load(os.path.join(DATADIR, 'train_arrays.npz'))

In [ ]:
train.files

In [ ]:
x_train = train['x']
meta_train = train['meta'].all().todense()
title_train = train['title'].all().todense()
desc_train = train['desc'].all().todense()
y_train = train['y'].all().todense()

print('x_train.shape = {}'.format(x_train.shape))
print('meta_train.shape = {}'.format(meta_train.shape))
print('title_train.shape = {}'.format(title_train.shape))
print('desc_train.shape = {}'.format(desc_train.shape))
print('y_train.shape = {}'.format(y_train.shape))

In [ ]:
dev = np.load(os.path.join(DATADIR, 'dev_arrays.npz'))

In [ ]:
x_dev = dev['x']
meta_dev = dev['meta'].all().todense()
title_dev = dev['title'].all().todense()
desc_dev = dev['desc'].all().todense()
y_dev = dev['y'].all().todense()

print('x_dev.shape = {}'.format(x_dev.shape))
print('meta_dev.shape = {}'.format(meta_dev.shape))
print('title_dev.shape = {}'.format(title_dev.shape))
print('desc_dev.shape = {}'.format(desc_dev.shape))
print('y_dev.shape = {}'.format(y_dev.shape))

In [ ]:
test = np.load(os.path.join(DATADIR, 'test_arrays.npz'))

In [ ]:
x_test = test['x']
meta_test = test['meta'].all().todense()
title_test = test['title'].all().todense()
desc_test = test['desc'].all().todense()
y_test = test['y'].all().todense()

print('x_test.shape = {}'.format(x_test.shape))
print('meta_test.shape = {}'.format(meta_test.shape))
print('title_test.shape = {}'.format(title_test.shape))
print('desc_test.shape = {}'.format(desc_test.shape))
print('y_test.shape = {}'.format(y_test.shape))

#### previous shapes in old data 
Shape of x_train: (150870, 1000)
Shape of metax_train: (150870, 436)
Shape of titlex_train: (150870, 10000)
Shape of descx_train: (150870, 10000)
Shape of y_train: (150870, 210)

Shape of x_dev: (9234, 1000)
Shape of meta_dev: (9234, 436)
Shape of titlex_dev: (9234, 10000)
Shape of descx_dev: (9234, 10000)
Shape of y_dev: (9234, 210)

Shape of x_test: (9234, 1000)
Shape of metax_test: (9234, 436)
Shape of titlex_test: (9234, 10000)
Shape of descx_test: (9234, 10000)
Shape of y_test: (9234, 210)

### preparing the Embedding layer


In [ ]:
import json
from collections import OrderedDict

def load_tokenizer_from_file(filename):
    
    tokenizer = Tokenizer()

    with open(filename, 'r') as infile:
        tokenizer_data = json.load(infile)

    tokenizer.word_counts = OrderedDict(tokenizer_data['word_counts'])
    tokenizer.word_docs = tokenizer_data['word_docs']
    tokenizer.word_index = tokenizer_data['word_index']
    tokenizer.document_count = tokenizer_data['document_count']
    tokenizer.index_docs = tokenizer_data['index_docs']

    return tokenizer

tokenizer_combined_text = load_tokenizer_from_file(os.path.join(DATADIR, "combined_text_tokenizer.json"))

In [ ]:
embedding_layer = Embedding(len(tokenizer_combined_text.word_index) + 1, 
                            EMBEDDING_DIM, 
                            input_length=MAX_SEQUENCE_LENGTH)

An Embedding layer should be fed sequences of integers, i.e. a 2D input of shape (samples, indices). These input sequences should be padded so that they all have the same length in a batch of input data (although an Embedding layer is capable of processing sequence of heterogenous length, if you don't pass an explicit input_length argument to the layer).

All that the Embedding layer does is to map the integer inputs to the vectors found at the corresponding index in the embedding matrix, i.e. the sequence [1, 2] would be converted to [embeddings[1], embeddings[2]]. This means that the output of the Embedding layer will be a 3D tensor of shape (samples, sequence_length, embedding_dim).

### Estimate class weights for unbalanced datasets.
paramter to model.fit = __class_weight__: Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.

Implement class_weight from sklearn:

- Import the module 

`from sklearn.utils import class_weight`
- calculate the class weight, If ‘balanced’, class weights will be given by n_samples / (n_classes * np.bincount(y)):

`class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)`

- change it to a dict in order to work with Keras.

`class_weight_dict = dict(enumerate(class_weight))`

- Add to model fitting

`model.fit(X_train, y_train, class_weight=class_weight)`

In [ ]:
# class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
# class_weight_dict = dict(enumerate(class_weight))

### Custom loss function

In [ ]:
class WeightedBinaryCrossEntropy(object):

    def __init__(self, pos_ratio):
        neg_ratio = 1. - pos_ratio
        #self.pos_ratio = tf.constant(pos_ratio, tf.float32)
        self.pos_ratio = pos_ratio
        #self.weights = tf.constant(neg_ratio / pos_ratio, tf.float32)
        self.weights = neg_ratio / pos_ratio
        self.__name__ = "weighted_binary_crossentropy({0})".format(pos_ratio)

    def __call__(self, y_true, y_pred):
        return self.weighted_binary_crossentropy(y_true, y_pred)

    def weighted_binary_crossentropy(self, y_true, y_pred):
            # Transform to logits
            epsilon = tf.convert_to_tensor(K.common._EPSILON, y_pred.dtype.base_dtype)
            y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
            y_pred = tf.log(y_pred / (1 - y_pred))

            cost = tf.nn.weighted_cross_entropy_with_logits(y_true, y_pred, self.weights)
            return K.mean(cost * self.pos_ratio, axis=-1)
    
y_true_arr = np.array([0,1,0,1], dtype="float32")
y_pred_arr = np.array([0,0,1,1], dtype="float32")
y_true = tf.constant(y_true_arr)
y_pred = tf.constant(y_pred_arr)

with tf.Session().as_default(): 
    print(WeightedBinaryCrossEntropy(0.5)(y_true, y_pred).eval())
    print(binary_crossentropy(y_true, y_pred).eval())


### difficulty getting global precision/recall metrics . CAUTION interpreting monitoring metrics
fcholltet: "Basically these are all global metrics that were approximated
batch-wise, which is more misleading than helpful. This was mentioned in
the docs but it's much cleaner to remove them altogether. It was a mistake
to merge them in the first place."

In [ ]:
def f1(y_true, y_pred):
    """Use Recall  and precision metrics to calculate harmonic mean (F1 score).

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1 = 2*((precision*recall)/(precision+recall))
    
    return f1

## Training a 1D convnet

### 1. Create model

In [ ]:
NB_CLASSES = y_train.shape[1]
NB_METAVARS = meta_train.shape[1]



sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='wordindex') #MAX_SEQUENCE_LENGTH
embedded_sequences = embedding_layer(sequence_input)
x = Dropout(0.2, name = 'dropout_embedded')(embedded_sequences)

x = Conv1D(128, 5, activation='relu', name = 'conv0')(x)

x = MaxPooling1D(5, name = 'max_pool0')(x)

x = Dropout(0.5, name = 'dropout0')(x)

x = Conv1D(128, 5, activation='relu', name = 'conv1')(x)

x = MaxPooling1D(5 , name = 'max_pool1')(x)

x = Conv1D(128, 5, activation='relu', name = 'conv2')(x)

x = MaxPooling1D(35, name = 'global_max_pool')(x)  # global max pooling

x = Flatten()(x) #reduce dimensions from 3 to 2; convert to vector + FULLYCONNECTED

meta_input = Input(shape=(NB_METAVARS,), name='meta')
meta_hidden = Dense(128, activation='relu', name = 'hidden_meta')(meta_input)
meta_hidden = Dropout(0.2, name = 'dropout_meta')(meta_hidden)


title_input = Input(shape=(title_train.shape[1],), name='titles')
title_hidden = Dense(128, activation='relu', name = 'hidden_title')(title_input)
title_hidden = Dropout(0.2, name = 'dropout_title')(title_hidden)

desc_input = Input(shape=(desc_train.shape[1],), name='descs')
desc_hidden = Dense(128, activation='relu', name = 'hidden_desc')(desc_input)
desc_hidden = Dropout(0.2, name = 'dropout_desc')(desc_hidden)

concatenated = concatenate([meta_hidden, title_hidden, desc_hidden, x])

x = Dense(400, activation='relu', name = 'fully_connected0')(concatenated)

x = Dropout(0.2, name = 'dropout1')(x)

x = Dense(NB_CLASSES, activation='sigmoid', name = 'fully_connected1')(x)

# # The Model class turns an input tensor and output tensor into a model
# This creates Keras model instance, will use this instance to train/test the model.
model = Model(inputs=[meta_input, title_input, desc_input, sequence_input], outputs=x)

In [ ]:
model.summary()

### Tensorboard callbacks /metrics /monitor training

<span style="color:red"> **Size of these files is killing storage during training. Is it histograms?**</span>

In [ ]:
tb = TensorBoard(log_dir='./learn_embedding_logs', histogram_freq=1, write_graph=True, write_images=False)

In [ ]:
CHECKPOINT_PATH = os.path.join(DATADIR, 'model_checkpoint.hdf5')

cp = ModelCheckpoint(
                     filepath = CHECKPOINT_PATH, 
                     monitor='val_loss', 
                     verbose=0, 
                     save_best_only=False, 
                     save_weights_only=False, 
                     mode='auto', 
                     period=1
                    )

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
#model.fit(x, y, validation_split=0.2, callbacks=[early_stopping])

### 2 & 3. Train & compile model

In [ ]:
# metrics callback causes: CCCCCCR55555555511155
# So disable for now
from keras.utils import multi_gpu_model

# Replicates `model` on 8 GPUs.
# This assumes that your machine has 8 available GPUs.
parallel_model = multi_gpu_model(model, gpus=8)
parallel_model.compile(loss=WeightedBinaryCrossEntropy(POS_RATIO),
              optimizer='rmsprop',
              metrics=['binary_accuracy', f1])

# This `fit` call will be distributed on 8 GPUs.
# Since the batch size is 256, each GPU will process 32 samples.
history = parallel_model.fit(
    {'meta': meta_train, 'titles': title_train, 'descs': desc_train, 'wordindex': x_train},
    y_train, 
    validation_data=([meta_dev, title_dev, desc_dev, x_dev], y_dev), 
    epochs=10, batch_size=128, callbacks=[early_stopping]
)



In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, 7)

plt.plot(epochs, loss_values, 'bo', label='Training loss')           
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')      
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()    

f1_values = history_dict['f1']
val_f1_values = history_dict['val_f1']

plt.plot(epochs, f1_values, 'bo', label='Training f1')
plt.plot(epochs, val_f1_values, 'b', label='Validation f1')
plt.title('Training and validation batch-level f1-micro')
plt.xlabel('Epochs')
plt.ylabel('F1-micro')
plt.legend()

plt.show()

### Save results arrays

In [ ]:
def to_file(array,name):
    df = pd.DataFrame(data = array.tolist(),columns=[i for i in range(1,y_train.shape[1]+1)])
    df.to_csv(os.path.join(DATADIR, name+'.csv.gz'),compression='gzip',index=False)

In [ ]:
date_run = time.strftime("_%H%M_%d%m_")
date_run

In [ ]:
y_prob = parallel_model.predict([meta_train, title_train, desc_train, x_train])

In [ ]:
to_file(y_prob,"train_results"+date_run)

In [ ]:
y_pred = y_prob.copy()
y_pred[y_pred>=P_THRESHOLD] = 1
y_pred[y_pred<P_THRESHOLD] = 0

In [ ]:
print('micro: {}'.format(precision_recall_fscore_support(y_train, y_pred, average='micro', sample_weight=None)))
print('macro: {}'.format(precision_recall_fscore_support(y_train, y_pred, average='macro', sample_weight=None)))
print('weightedmacro: {}'.format(precision_recall_fscore_support(y_train, y_pred, average='weighted', sample_weight=None)))

In [ ]:
y_prob_dev = parallel_model.predict([meta_dev, title_dev, desc_dev, x_dev])

In [ ]:
to_file(y_prob_dev,"dev_results"+date_run)

In [ ]:
y_pred_dev = y_prob_dev.copy()
y_pred_dev[y_pred_dev>=P_THRESHOLD] = 1
y_pred_dev[y_pred_dev<P_THRESHOLD] = 0

In [ ]:
print('micro: {}'.format(precision_recall_fscore_support(y_dev, y_pred_dev, average='micro', sample_weight=None)))
print('macro: {}'.format(precision_recall_fscore_support(y_dev, y_pred_dev, average='macro', sample_weight=None)))
print('weightedmacro: {}'.format(precision_recall_fscore_support(y_dev, y_pred_dev, average='weighted', sample_weight=None)))

In [ ]:
print('weightedmacro: {}'.format(precision_recall_fscore_support(y_dev, y_pred_dev, average=None, sample_weight=None)))

In [ ]:
to_file(y_train,"true_train"+date_run)

In [ ]:
to_file(y_dev,"true_dev"+date_run)

### predictions

In [ ]:
predict_filename = "predict_arrays.npz"
predict = np.load(os.path.join(DATADIR, predict_filename))
get_predictions(predict)

In [ ]:
id_predict = predict['content_id']
id_predict[0:10]

In [ ]:
def get_predictions(arrays):  
    
    print('Set up arrays for new_content: {}'.format(arrays.files))
    x_predict = arrays['x']
    meta_predict = arrays['meta'].all().todense()
    title_predict = arrays['title'].all().todense()
    desc_predict = arrays['desc'].all().todense()
    
    print('x_arrays.shape = {}'.format(x_predict.shape))
    print('meta_arrays.shape = {}'.format(meta_predict.shape))
    print('title_arrays.shape = {}'.format(title_predict.shape))
    print('desc_arrays.shape = {}'.format(desc_predict.shape))
    
    print('Predict on untagged content')
    y_pred_new = model.predict([meta_predict, title_predict, desc_predict, x_predict])
    
    to_file(y_pred_new, "new_predictions"+date_run)

In [ ]:
get_predictions(predict)

In [ ]:
test_change